In [1]:
import refinitiv.data as rd
from refinitiv.data.content import historical_pricing as hp


In [3]:
"""
#Start Refinitiv Session
rd.open_session(app_key="2ad05903f7cb4f84b75e13735fb71abde98f7d85")
"""

'\n#Start Refinitiv Session\nrd.open_session(app_key="2ad05903f7cb4f84b75e13735fb71abde98f7d85")\n'

In [5]:
"""
#Define instruments and Date range
START_DATE = "2006-01-01"
END_DATE = "2025-06-30"

# Replace these placeholders with the actual Refinitiv RICs 
SPX_PROXY_RIC     = "SPY.P"   # S&P 500 ETF (proxy for .SPX)
DOLLAR_INDEX_RIC  = ".DXY"    # Dollar Index
WTI_OIL_RIC       = "CLc1"    # WTI Crude Oil Futures (continuous contract)
"""

'\n#Define instruments and Date range\nSTART_DATE = "2006-01-01"\nEND_DATE = "2025-06-30"\n\n# Replace these placeholders with the actual Refinitiv RICs \nSPX_PROXY_RIC     = "SPY.P"   # S&P 500 ETF (proxy for .SPX)\nDOLLAR_INDEX_RIC  = ".DXY"    # Dollar Index\nWTI_OIL_RIC       = "CLc1"    # WTI Crude Oil Futures (continuous contract)\n'

In [7]:
#import refinitiv.data as rd
#print("Refinitiv Data Library version:", rd.__version__)

#print(dir(hp))

"""
from refinitiv.data.content import pricing

# request latest snapshot for Apple
snap = pricing.Definition(universe="AAPL.O").get_data()
print(snap)
"""

"""
#Obtain required data
# --- 1. S&P 500 OHLCV ---
df_spx_raw = hp.summaries.Definition(
    universe=SPX_PROXY_RIC,
    fields=["OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"],
    interval=hp.Intervals.DAILY,
    start=START_DATE,
    end=END_DATE
).get_data()

# --- 2. Dollar Index & WTI Oil (Closing Prices) ---
df_macro_raw = hp.summaries.Definition(
    universe=[DOLLAR_INDEX_RIC, WTI_OIL_RIC],
    fields=["CLOSE"],
    interval=hp.Intervals.DAILY,
    start=START_DATE,
    end=END_DATE
).get_data()

# Output checks
print("S&P 500 Raw DataFrame (df_spx_raw):")
print(df_spx_raw.head())

print("\nMacro Raw DataFrame (df_macro_raw):")
print(df_macro_raw.head())
"""

'\n#Obtain required data\n# --- 1. S&P 500 OHLCV ---\ndf_spx_raw = hp.summaries.Definition(\n    universe=SPX_PROXY_RIC,\n    fields=["OPEN", "HIGH", "LOW", "CLOSE", "VOLUME"],\n    interval=hp.Intervals.DAILY,\n    start=START_DATE,\n    end=END_DATE\n).get_data()\n\n# --- 2. Dollar Index & WTI Oil (Closing Prices) ---\ndf_macro_raw = hp.summaries.Definition(\n    universe=[DOLLAR_INDEX_RIC, WTI_OIL_RIC],\n    fields=["CLOSE"],\n    interval=hp.Intervals.DAILY,\n    start=START_DATE,\n    end=END_DATE\n).get_data()\n\n# Output checks\nprint("S&P 500 Raw DataFrame (df_spx_raw):")\nprint(df_spx_raw.head())\n\nprint("\nMacro Raw DataFrame (df_macro_raw):")\nprint(df_macro_raw.head())\n'